In [5]:
import os
import time
import datetime
import pytz
import json
import time

import boto3
import base64

In [11]:
# get the timestamp for the build
format = "%Y%m%d%H%M%S"
timestamp = time.strftime(format)

print("time stamp is: {}".format(timestamp))

# DB settings
schema_name_dev = "biodev"
schema_name_new = 'bioindex-' + timestamp

# s3 settings
s3_bucket_new = 'dig-bio-index-' + timestamp
s3_bucket_dev = 'dig-bio-index-dev'

# git settings
code_directory = '/Users/mduby/BioIndex/'
git_directory = code_directory + 'bioindex_' + timestamp
git_clone_command = "git clone git@github.com:broadinstitute/dig-bioindex.git " + git_directory

# secrets settings
secret_name = "bioindex-dev"
region_name = "us-east-1"

# log
print("got initial schema name: {} and s3 index: {} and git directory {}".format(schema_name_dev, s3_bucket_dev, git_directory))
print("the git clone command is: {}".format(git_clone_command))


time stamp is: 20200424092216
got initial schema name: biodev and s3 index: dig-bio-index-dev and git directory /Users/mduby/BioIndex/bioindex_20200424092216
the git clone command is: git clone git@github.com:broadinstitute/dig-bioindex.git /Users/mduby/BioIndex/bioindex_20200424092216


In [85]:
# method to run an OS command and time it
def run_system_command(os_command, input_message = "", if_test = True, if_log = False):
    log_message = "Running command"
    exit_code = None
    start = time.time()
    if if_test:
        log_message = "Testing command"
    if if_log:
        print("{}: {}".format(log_message, os_command))
    if not if_test:
        exit_code = os.system(os_command)
    end = time.time()
    print("Done in {}s with exit code {}".format(end - start, exit_code))


In [12]:
# log
print("Running git clone command: {}".format(git_clone_command))

# clone the bioindex
os.chdir(code_directory)
clone_output = os.system(git_clone_command)

# log
print("the git process exited with code {}".format(clone_output))

Running git clone command: git clone git@github.com:broadinstitute/dig-bioindex.git /Users/mduby/BioIndex/bioindex_20200424092216
the git process exited with code 0


In [13]:
# list buckets
s3client = boto3.client('s3')

# print all the bucket names
list_buckets_resp = s3client.list_buckets()
for bucket in list_buckets_resp['Buckets']:
    if 'index' in bucket['Name']:
        print("existing bucket name: {}".format(bucket['Name']))



existing bucket name: dig-bio-index
existing bucket name: dig-bio-index-20200421132912
existing bucket name: dig-bio-index-20200421135919
existing bucket name: dig-bio-index-20200421152915
existing bucket name: dig-bio-index-dev


In [15]:
# create the s3 bucket
s3client.create_bucket(Bucket=s3_bucket_new)

list_buckets_resp = s3client.list_buckets()
for bucket in list_buckets_resp['Buckets']:
    if bucket['Name'] == s3_bucket_new:
        print('(Just created) --> {} - there since {}'.format(bucket['Name'], bucket['CreationDate']))

(Just created) --> dig-bio-index-20200424092216 - there since 2020-04-24 13:53:56+00:00


In [16]:
# print all the bucket names
list_buckets_resp = s3client.list_buckets()
for bucket in list_buckets_resp['Buckets']:
    if 'index' in bucket['Name']:
        print("existing bucket name after addition: {}".format(bucket['Name']))


existing bucket name after addition: dig-bio-index
existing bucket name after addition: dig-bio-index-20200421132912
existing bucket name after addition: dig-bio-index-20200421135919
existing bucket name after addition: dig-bio-index-20200421152915
existing bucket name after addition: dig-bio-index-20200424092216
existing bucket name after addition: dig-bio-index-dev


In [86]:
# clone the bucket
# s3_all_objects = s3client.list_objects_v2(Bucket = s3_bucket_bio_index) 
session = boto3.session.Session()
s3 = session.resource('s3')
s3_new_bucket = s3.Bucket(s3_bucket_dev)

# s3_new_bucket = s3client.Bucket(s3_bucket_name)

# for s3object in s3_all_objects.get('Contents'):
#     print("the dev s3 bioindex bucket {} has sub object: {}".format(s3_bucket_bio_index, s3object))

# for s3object in s3_new_bucket.objects.filter(Prefix = "/"):
#     print("the dev s3 bioindex bucket {} has sub object: {}".format(s3_bucket_bio_index, s3object))    

# log
print("sub folders of {} that need to be cloned".format(s3_bucket_dev))
result = s3client.list_objects(Bucket=s3_bucket_dev, Prefix="", Delimiter='/')
for s3object in result.get('CommonPrefixes'):
    print("-> sub folder: {}".format(s3object.get('Prefix')))

# log
print("\ncloning s3 bucket {}".format(s3_bucket_dev))

# copy the data
for s3object in result.get('CommonPrefixes'):
    s3_subdirectory = s3object.get('Prefix')
    s3_command = "aws s3 sync s3://{}/{} s3://{}/{}".format(s3_bucket_dev, s3_subdirectory, s3_bucket_new, s3_subdirectory)
    run_system_command(s3_command, if_log= True)
    # print("s3 copying command: {}".format(s3_command))
    # exit_code = os.system(s3_command)
    # end = time.time()
    # print("Done in {}s with exit code {}".format(end - start, exit_code))


sub folders of dig-bio-index-dev that need to be cloned
-> sub folder: associations/
-> sub folder: burden/
-> sub folder: genes/

cloning s3 bucket dig-bio-index-dev
Testing command: aws s3 sync s3://dig-bio-index-dev/associations/ s3://dig-bio-index-20200424092216/associations/
Done in 4.1961669921875e-05s with exit code None
Testing command: aws s3 sync s3://dig-bio-index-dev/burden/ s3://dig-bio-index-20200424092216/burden/
Done in 3.790855407714844e-05s with exit code None
Testing command: aws s3 sync s3://dig-bio-index-dev/genes/ s3://dig-bio-index-20200424092216/genes/
Done in 6.914138793945312e-05s with exit code None


In [3]:
# Use this code snippet in your app.
# If you need more information about configurations or implementing the sample code, visit the AWS docs:   
# https://aws.amazon.com/developers/getting-started/python/

import boto3
import base64
from botocore.exceptions import ClientError


def get_secret():

    secret_name = "bioindex-dev"
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    # In this sample we only handle the specific exceptions for the 'GetSecretValue' API.
    # See https://docs.aws.amazon.com/secretsmanager/latest/apireference/API_GetSecretValue.html
    # We rethrow the exception by default.

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        if e.response['Error']['Code'] == 'DecryptionFailureException':
            # Secrets Manager can't decrypt the protected secret text using the provided KMS key.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'InternalServiceErrorException':
            # An error occurred on the server side.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'InvalidParameterException':
            # You provided an invalid value for a parameter.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'InvalidRequestException':
            # You provided a parameter value that is not valid for the current state of the resource.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'ResourceNotFoundException':
            # We can't find the resource that you asked for.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
    else:
        # Decrypts secret using the associated KMS CMK.
        # Depending on whether the secret is a string or binary, one of these fields will be populated.
        if 'SecretString' in get_secret_value_response:
            secret = get_secret_value_response['SecretString']
        else:
            decoded_binary_secret = base64.b64decode(get_secret_value_response['SecretBinary'])

    return json.loads(secret)

In [80]:
# get the secret
bio_secret = get_secret()

print("got secret of type {}".format(type(bio_secret)))
print("the secret name is: {}".format(bio_secret['dbInstanceIdentifier']))

got secret of type <class 'dict'>
the secret name is: dig-bio-index


In [87]:
# build the mysql command
mysql_user = bio_secret['username']
mysql_password = bio_secret['password']
mysql_host = bio_secret['host']
mysql_db = bio_secret['dbname']

# build the create the database
# mysql_command_create_schema = "mysql -u {} -p'{}' -e \"create database {}\"".format(mysql_user, mysql_password, schema_name_new)

# run the command
run_system_command(mysql_command_create_schema)

Done in 7.152557373046875e-07s with exit code None


In [88]:
# build the mysql schema cloning command
mysql_command_dump = "mysqldump -u {} -p'{}' -h {} {}".format(mysql_user, mysql_password, mysql_host, mysql_db)
mysql_command_load = "mysql -u {} -p'{}' -h {} {}".format(mysql_user, mysql_password, mysql_host, schema_name_new)
mysql_command_combined = mysql_command_dump + " | " + mysql_command_load

# print
# print("mysql dump command: {}".format(mysql_command_dump))
# print("mysql load command: {}".format(mysql_command_load))

# run the schema clone command
run_system_command(mysql_command_combined)



Done in 9.5367431640625e-07s with exit code None


In [51]:
# update the secret
session = boto3.session.Session()
client = session.client(
    service_name='secretsmanager',
    region_name=region_name
)

updated_secret = bio_secret.update({"s3bucket": s3_bucket_name})
client.update_secret(SecretId=secret_name, SecretString=json.dumps(bio_secret))



{'ARN': 'arn:aws:secretsmanager:us-east-1:005901288866:secret:bioindex-dev-oN7jhA',
 'Name': 'bioindex-dev',
 'VersionId': '5a654c67-0ef5-42ed-b407-d010c038d0ac',
 'ResponseMetadata': {'RequestId': '779c1e36-80e2-4e21-a63b-dda77d870d15',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 21 Apr 2020 16:10:08 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '155',
   'connection': 'keep-alive',
   'x-amzn-requestid': '779c1e36-80e2-4e21-a63b-dda77d870d15'},
  'RetryAttempts': 0}}

In [40]:
secret_name = "bioindex-new-create"
region_name = "us-east-1"

# Create a Secrets Manager client
session = boto3.session.Session()
client = session.client(
    service_name='secretsmanager',
    region_name=region_name
)

client.create_secret(Name = secret_name)

{'ARN': 'arn:aws:secretsmanager:us-east-1:005901288866:secret:bioindex-new-create-RjrVAj',
 'Name': 'bioindex-new-create',
 'ResponseMetadata': {'RequestId': '9159a207-bc44-4e4e-b7af-5478a6ac6873',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 24 Apr 2020 19:54:23 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '118',
   'connection': 'keep-alive',
   'x-amzn-requestid': '9159a207-bc44-4e4e-b7af-5478a6ac6873'},
  'RetryAttempts': 0}}